# Get predictions for new data

In [ ]:
import pycaret 
print(pycaret.__version__)

In [ ]:
import os, uuid
from azure.storage.blob import BlobServiceClient, BlobClient, ContainerClient
from pycaret.regression import *

In [ ]:
def download_blob(container_name, source_blob_name, destination_file_name):
  # Download the blob to a local file
    print("\nDownloading blob to \n\t" + destination_file_name)

  # Create a blob client using the local file name as the name for the blob
    blob_client = blob_service_client.get_blob_client(container=container_name, blob=source_blob_name)

    if destination_file_name is not None: 
        with open(destination_file_name, "wb") as download_file:
            download_file.write(blob_client.download_blob().readall())

        print(
            "Blob {} downloaded to {}.".format(
                source_blob_name, destination_file_name
            )
        )

#### We establish the connection

In [ ]:
## Enter connection string when running in google colab
connect_str = '' 
print(connect_str)

# Create the BlobServiceClient object which will be used to create a container client
blob_service_client = BlobServiceClient.from_connection_string(connect_str)

In [ ]:
# Create a unique name for the container
container_name = "machinelearninginsurance" 
model_name = 'lightgbm-reg-101'

##### Model Download

In [ ]:
outfile_name = 'lightgbm-reg101-azure-downloaded'
model_azure_src = str(model_name)+'.pkl'
download_blob(container_name, model_azure_src, outfile_name + '.pkl')

### Forecast with new data

In [ ]:
# Loading the model for predictions
azure_final_lightgbm = load_model(outfile_name)

In [ ]:
from pycaret.datasets import get_data
dataset = get_data('diamond')

In [ ]:
data = dataset.sample(frac=0.95, random_state=786).reset_index(drop=True)
data_unseen = dataset.drop(data.index).reset_index(drop=True)

In [ ]:
# Predictions from deployed model
new_prediction_azure = predict_model(azure_final_lightgbm, data=data_unseen)
new_prediction_azure.head()